In [ ]:
!pip install fasttext




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199772 sha256=2261af12dcf462835037d756cf75dd3ee229efc8fb308427814c5c0249ab3020
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import fasttext
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving multichannel_bsentiment_train.xlsx to multichannel_bsentiment_train (1).xlsx


In [ ]:
df = pd.read_excel("multichannel_bsentiment_train.xlsx")

# Display the first few rows of the DataFrame
print(df)

        id                                               text class_label
0      473                    আল্লাহ অবশ্যই এদের পাশে থাকবে।     Positive
1     1954                     বেগম খালেদা জিয়া সঠিক বলেছেন।     Positive
2     2145   ভারতীয় অনূর্ধ্ব-১৯ দলের কোচ ম্যাচ শেষে পাকিস্...    Positive
3     2250  খুব গুরুত্ব পূর্ন একটি ভাল সিদ্ধান্ত রাজ্জাক ভ...    Positive
4     4680   এই বাভে যদি সফর করা যেত মানুষ আর লক্ষ লক্ক টা...    Negative
...    ...                                                ...         ...
5505  4762        খানকিরা প্রভাবশালী পৃথিবী কোন দিকে যাচ্ছে?     Negative
5506  2608                         এখানে ভাজা চাল আশ্চর্যজনক।    Positive
5507  6805            বাংলাদেশের সাপোর্ট পাওয়ার জন্য ধন্যবাদ,    Negative
5508  4030   দেখেই বুঝা যাচ্ছে এইবার আর সাস্থ্যর কোনো সমস্...    Negative
5509  2586                  আমি সত্যিই এই জায়গা পছন্দ করেছি।    Positive

[5510 rows x 3 columns]


In [ ]:
def load_data(df):
    # Your data loading logic here
    # Assuming 'text' column contains Bengali text and 'class_label' contains sentiment labels
    return X, y

In [ ]:
df = pd.read_excel("multichannel_bsentiment_train.xlsx")


X, y = load_data(df)
df


,id,text,class_label
0,473,আল্লাহ অবশ্যই এদের পাশে থাকবে।,Positive
1,1954,বেগম খালেদা জিয়া সঠিক বলেছেন।,Positive
2,2145,ভারতীয় অনূর্ধ্ব-১৯ দলের কোচ ম্যাচ শেষে পাকিস্...,Positive
3,2250,খুব গুরুত্ব পূর্ন একটি ভাল সিদ্ধান্ত রাজ্জাক ভ...,Positive
4,4680,এই বাভে যদি সফর করা যেত মানুষ আর লক্ষ লক্ক টা...,Negative
...,...,...,...
5505,4762,খানকিরা প্রভাবশালী পৃথিবী কোন দিকে যাচ্ছে?,Negative
5506,2608,এখানে ভাজা চাল আশ্চর্যজনক।,Positive
5507,6805,"বাংলাদেশের সাপোর্ট পাওয়ার জন্য ধন্যবাদ,",Negative
5508,4030,দেখেই বুঝা যাচ্ছে এইবার আর সাস্থ্যর কোনো সমস্...,Negative


In [ ]:
X = df['text'].values
y = df['class_label'].values
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# TF-IDF feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
# Print some information about the datasets
print("\nTraining Set Size:", len(X_train))
print("Testing Set Size:", len(X_test))
print("Unique Classes in Training Set:", np.unique(y_train))


Training Set Size: 4408
Testing Set Size: 1102
Unique Classes in Training Set: ['Negative' 'Positive']


In [ ]:
# Save the TF-IDF data in FastText format
def save_data_fasttext_tfidf(X_tfidf, y, df):
    with open(df, 'w', encoding='utf-8') as f:
        for text, label in zip(X_tfidf, y):
            f.write(f'__label__{label} {text}\n')
save_data_fasttext_tfidf(X_train_tfidf, y_train, 'train_data_tfidf.txt')
save_data_fasttext_tfidf(X_test_tfidf, y_test, 'test_data_tfidf.txt')


In [ ]:
# Train the FastText model
model = fasttext.train_supervised(input='train_data_tfidf.txt', epoch=10, lr=0.1, wordNgrams=2)



In [ ]:
y_pred_prob = []
for text in X_test:
    prob = model.predict(text)[1][0]
    y_pred_prob.append(prob)
threshold = 0.5
y_pred_binary = (np.array(y_pred_prob) > threshold).astype(int)

# Convert string labels to integers in y_test using numpy.vectorize
label_mapping = {'Negative': 0, 'Positive': 1}
vectorized_mapping = np.vectorize(lambda x: label_mapping[x])
y_test_int = vectorized_mapping(y_test)

# Calculate precision, recall, and confusion matrix
precision = precision_score(y_test_int, y_pred_binary)
recall = recall_score(y_test_int, y_pred_binary)
conf_matrix = confusion_matrix(y_test_int, y_pred_binary)

# Print results
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print('Confusion Matrix:')
print(conf_matrix)

# Calculate and print accuracy and classification report
accuracy = accuracy_score(y_test_int, y_pred_binary)
class_report = classification_report(y_test_int, y_pred_binary)
print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', class_report)







Precision: 0.4501
Recall: 1.0000
Confusion Matrix:
[[  0 606]
 [  0 496]]
Accuracy: 0.4501
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       606
           1       0.45      1.00      0.62       496

    accuracy                           0.45      1102
   macro avg       0.23      0.50      0.31      1102
weighted avg       0.20      0.45      0.28      1102



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
for line in X_train[:5]:
    print(line)


 যত তাড়াতাড়ি সম্ভব এর একটা সমাধান চাই 
 এটা হাছিনার দ্বারাই সম্ভব হলেও হইতে পারে হুম কম খারাপ না আইডিয়াটা। 
রাজ্জাক রাজ্জাকের মতোই ফিরে আসছে।
 তোদের ল্যাংটাচোদা ভারতকে বাংলাদেশ দান করেছে । ভিখারি চোদা তুই তো তাই ভিখারি ভাষায় কথা বলিস । আমাদের দেশের খরাতের দেওয়া টাকায় তোদের রেন্ডিয়া চলে । 
 আপনাদের লাঠিবাহিনি আছে তাই সেনাবাহিনীর দকরার নাই 
